In [1]:
from langchain_openai import ChatOpenAI
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, AgentType
from pydantic import BaseModel, Field
from typing import Any, Type
from langchain_community.tools import WikipediaQueryRun, GoogleSearchRun
from langchain_community.utilities import WikipediaAPIWrapper, GoogleSearchAPIWrapper

In [2]:
llm = ChatOpenAI(
    temperature=0.1,
    model_name="gpt-4o-mini",
)

In [3]:
class WikipediaSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for Example query: Search for Apple Company."
    )


class WikipediaSearchTool(BaseTool):
    name: str = "WikipediaSearchTool"
    description: str = (
        "Use this tool to find any information for the query. It takes a query as an argument. And save the results in file."
    )
    args_schema: Type[WikipediaSearchToolArgsSchema] = WikipediaSearchToolArgsSchema

    def _run(self, query):
        wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
        return wikipedia.run(query)

In [4]:
class SearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search websites about Example query: Search websites about Apple Company."
    )


class SearchTool(BaseTool):
    name: str = "SearchTool"
    description: str = (
        "Use this tool to find websites relate about the query. It takes a query as an argument. And save the results in file."
    )
    args_schema: Type[SearchToolArgsSchema] = SearchToolArgsSchema

    def _run(self, query):
        google = GoogleSearchAPIWrapper()
        return google.run(query)

In [5]:
class SaveToolArgsSchema(BaseModel):
    filename: str = Field(description="Filename to save the search results.")
    contents: str = Field(description="the search results.")


class SaveTool(BaseTool):
    name: str = "SaveTool"
    description: str = "Use this tool to save search results in text file."
    args_schema: Type[SaveToolArgsSchema] = SaveToolArgsSchema

    def _run(self, filename, contents):
        # print(filename, contents)
        f = open(filename, "w", encoding="utf-8")
        f.write(contents)
        f.close()

In [ ]:
agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        WikipediaSearchTool(),
        SearchTool(),
        SaveTool(),
    ],
)

prompt = "Research about the XZ backdoor"

agent.invoke(prompt)